## PET-CT Lymphoma Segmentation with nnUNet

This notebook demonstrates how to use nnUNet for PET-CT Lymphoma Segmentation. We will use the nnUNetV2Runner MONAI implmentation to simplify the training process.

In [3]:
import os
import random
from monai.bundle.config_parser import ConfigParser
from monai.apps.nnunet import nnUNetV2Runner

from pathlib import Path
import json

/home/maia-user/.conda/envs/MONAI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataroot = "Data"

In [5]:
test_dir = os.path.join(dataroot,"Task100_AutoPET_Lymphoma", "imagesTs/")
train_dir = os.path.join(dataroot,"Task100_AutoPET_Lymphoma", "imagesTr/")
label_dir = os.path.join(dataroot,"Task100_AutoPET_Lymphoma", "labelsTr/")

In [6]:
datalist_json = {"testing": [], "training": []}

In [7]:
datalist_json["testing"] = [
    {"image": "./imagesTs/" + file}
    for file in os.listdir(test_dir)
    if ("_0000.nii.gz" in file) and ("._" not in file)
]

In [8]:
datalist_json["training"] = [
{"image": "./imagesTr/" + file, "fold": 0, "label": "./labelsTr/" + file.replace("_0000",""),}    
for file in os.listdir(train_dir)
    if (".nii.gz" in file) and ("._" not in file)
]  # Initialize as single fold

In [9]:
random.seed(42)
random.shuffle(datalist_json["training"])

In [10]:
num_folds = 5
fold_size = len(datalist_json["training"]) // num_folds
for i in range(num_folds):
    for j in range(fold_size):
        datalist_json["training"][i * fold_size + j]["fold"] = i

In [11]:
datalist_file = Path(dataroot).joinpath("Task100_AutoPET_Lymphoma","Task100_AutoPET_Lymphoma_folds.json")
with open(datalist_file, "w", encoding="utf-8") as f:
    json.dump(datalist_json, f, ensure_ascii=False, indent=4)
print(f"Datalist is saved to {datalist_file}")

Datalist is saved to /home/maia-user/Documents/Data/Task100_AutoPET_Lymphoma/Task100_AutoPET_Lymphoma_folds.json


In [12]:
nnunet_root_dir = os.path.join(dataroot, "nnUNet")

os.makedirs(nnunet_root_dir, exist_ok=True)

data_src_cfg = os.path.join(nnunet_root_dir, "data_src_cfg.yaml")
data_src = {
    "modality": ["CT","PET"],
    "dataset_name_or_id": "100",
    "datalist": os.path.join(dataroot, "Task100_AutoPET_Lymphoma/Task100_AutoPET_Lymphoma_folds.json"),
    "dataroot": os.path.join(dataroot, "Task100_AutoPET_Lymphoma"),
}

ConfigParser.export_config_file(data_src, data_src_cfg)

In [13]:
runner = nnUNetV2Runner(
    input_config=data_src_cfg, trainer_class_name="nnUNetTrainer", work_dir=nnunet_root_dir
)

In [14]:
runner.convert_dataset()

2025-02-21 13:14:11,363 - WARNING - Dataset with the same ID exists!


In [16]:
runner.plan_and_process(c= ["3d_fullres"],n_proc=[1],verify_dataset_integrity=True,npfp=1, verbose=True)

2025-02-21 13:22:46,765 - INFO - Fingerprint extraction...
Dataset100_Task100_AutoPET_Lymphoma
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

2025-02-21 13:28:55,757 - INFO - Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Attempting to find 3d_lowres config. 
Current spacing: [3.09       2.09751271 2.09751271]. 
Current patch size: (128, 128, 128). 
Current median shape: [314.5631068  388.34951456 388.34951456]
Attempting to find 3d_lowres config. 
Current spacing: [3.1827     2.16043809 2.16043809]. 
Current patch size: (128, 128, 128). 
Current 